In [ ]:
import pandas as pd
import geopandas as gpd
import json
import matplotlib.pyplot as plt
%matplotlib inline

from bokeh.io import output_file, show, output_notebook, export_png
from bokeh.models import ColumnDataSource, GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.plotting import figure
from bokeh.palettes import brewer

import panel as pn
import panel.widgets as pnw
from bokeh.models.widgets import DataTable
import datetime as dt
from bokeh.tile_providers import CARTODBPOSITRON, get_provider

In [ ]:
output_notebook()

In [ ]:
data = gpd.read_file('tmp_death/20-04-24_right.geojson')
data.geometry = data.geometry.simplify(0.01)

In [ ]:
data = data[data['likelihood_cluster'].notna()]

In [ ]:
data

In [ ]:
fig, ax = plt.subplots(1, figsize=(14, 8))
data.plot(column='likelihood_cluster', cmap='OrRd', linewidth=0.2, ax=ax, edgecolor='black')
ax.axis('off')
ax.set_title('Hi - Clusters in 2020-05-19', fontsize=18)

In [ ]:
def get_geodatasource(gdf):    
    """Get getjsondatasource from geopandas object"""
    json_data = json.dumps(json.loads(gdf.to_json()))
    return GeoJSONDataSource(geojson = json_data)

geosource = get_geodatasource(data)

In [ ]:
tile_provider = get_provider(CARTODBPOSITRON)

In [ ]:
def bokeh_plot_map(column=None, title=''):
    """Plot bokeh map from GeoJSONDataSource """
    column = 'likelihood_cluster'
    palette = brewer['OrRd'][9]
    palette = palette[::-1]
    vals = data[column]
    #Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
    color_mapper = LinearColorMapper(palette = palette, low = vals.min(), high = vals.max())
    color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=500, height=20,
                         location=(0,0), orientation='horizontal')

    tools = 'wheel_zoom,pan,reset'
    p = figure(title = title, plot_height=400 , plot_width=850, toolbar_location='right', tools=tools)
    p.add_tile(tile_provider)
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    #Add patch renderer to figure
    p.patches('xs','ys', source=geosource, fill_alpha=0.8, line_width=0.5, line_color='black',  
              fill_color={'field' :column , 'transform': color_mapper})
    #Specify figure layout.
    p.add_layout(color_bar, 'below')
    
    return p

In [ ]:
def map_dash():
    """Map dashboard"""
    map_pane = pn.pane.Bokeh(width=800)
#     data_select = pnw.Select(name='dataset',options=list(owid.index))
    date_slider = pnw.DateSlider(name='Date', 
                                 start=dt.datetime(2020, 5, 18), 
                                 end=dt.datetime(2020, 5, 19), 
                                 value=dt.datetime(2020, 5, 19))
#     year_slider = pnw.IntSlider(start=1950,end=2018,value=2010)
    def update_map(event):    
        map_pane.object = bokeh_plot_map(date_slider.value)        
        return
    date_slider.param.watch(update_map,'value')
    date_slider.param.trigger('value')
#     data_select.param.watch(update_map,'value')
    app = pn.Column(pn.Row(date_slider),map_pane)
    return app

app = map_dash()

In [ ]:
a = bokeh_plot_map(dt.datetime(2020, 4, 24))

In [ ]:
show(a)

In [ ]:
show(a)

In [ ]:
pn.extension()

In [ ]:
app